In [1]:
import scipy
from scipy import io
import matplotlib.pyplot as plt
import cv2
import glob
import os
from numpy.linalg import lstsq
import numpy as np
import tensorflow as tf
import sklearn
from sklearn import utils
import numpy as np
from matplotlib import pyplot as plt
from patchify import patchify
import tifffile as tiff
import pandas as pd
import torch

import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt

import tensorflow as tf
import os
import random
import numpy as np
from tqdm import tqdm 
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
import keras
from patchify import patchify, unpatchify
from keras import utils
import tensorflow as tf
import os
import random
import numpy as np
from tqdm import tqdm 
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
import keras
from patchify import patchify, unpatchify
from keras import utils
import skimage

from skimage import measure, io, img_as_ubyte
import matplotlib.pyplot as plt
from skimage.color import label2rgb, rgb2gray
import numpy as np
import gc
from skimage import measure, color, io
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda


from skimage import measure, io, img_as_ubyte
import networkx as nx
import matplotlib.pyplot as plt
from skimage.color import label2rgb, rgb2gray
import numpy as np
import gc
from skimage.feature import graycomatrix, graycoprops
from skimage import measure, color, io
from keras.models import Model
from itertools import repeat
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda

/Users/cobeliu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import LinearSVC
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score

In [3]:
def get_edge_list(points, distance):
    edge_list_1st_row = []
    edge_list_2nd_row = []
    dist_list = []
    for i in range(len(points)):
        dist = np.sqrt((points[i,0] - points[:,0])**2 + (points[i,1] - points[:,1])**2)
        dist = dist.reshape(-1,1)
        #print(dist)
        x = np.where(dist<=distance)
        node_index = list(x[0])
        dist_list.append(np.average(dist[node_index]))
        edge_list_2nd_row.append(node_index)
        edge_list_1st_row.extend(repeat(i,len(node_index)))
    edge_list_2nd_row = [item for sublist in edge_list_2nd_row for item in sublist]
    edge_list = [edge_list_1st_row, edge_list_2nd_row]
    edge_list = np.array(edge_list)
    dist_list = np.array(dist_list)
    dist_list = dist_list.reshape(-1,1)
    return edge_list, dist_list

In [4]:
def get_graph(edge_list):
    G = nx.Graph()
    G.add_edges_from(edge_list.T)
    return G

In [5]:
def get_eccent(G, bool_get_90=True, verbose = True):
    temp_gen = nx.shortest_path_length( G)
    short_path_len = [list(temp[1].values()) for temp in temp_gen]
    try:
        avg_short_path = nx.average_shortest_path_length(G)
    except:
        avg_short_path = np.inf
        
    eccent_L = []
        
    if bool_get_90:
        eccent_90_L = []
        
    for sub_len_list in short_path_len:
        eccent_L.append(np.max(sub_len_list))
            
        if bool_get_90:
            eccent_90_L.append(np.percentile(sub_len_list, 90, overwrite_input = True))
    
    eccent = np.array( eccent_L)
    if bool_get_90:
        eccent_90 = np.array( eccent_90_L) 
        
    return eccent, eccent_90, avg_short_path

In [6]:
def _get_slope( eig, low, high):
    
    eig_abs = np.sort(eig)
    idx = np.where( (eig_abs>=low) & (eig_abs<high))[0]
    x = np.arange( idx.shape[0])
    y = eig_abs[idx]
    
    m,c = fit_line_mul( x, y)
    
    return m

In [7]:
def get_feats_spect( G, eps = 1e-10, 
                    library = 0, 
                    bool_eig_symm = True):
    
    #"Global Graph Features for Spectral Measures:"
    #available:  [20, 21, 23, 25, 26, 27, 28, 29, 30]
    #omitted:    [22, 24]
    
    
    #Metrics related to adjacency matrix
    
    if library == 1:
        A = np.array(G.get_adjacency().data)
    else:
        A = nx.adjacency_matrix(G).toarray()
        
    if not bool_eig_symm:
        eig = scipy.linalg.eig( A, right = False).astype(float)        
    else:        
        eig = scipy.linalg.eigh( A, eigvals_only = True)
    
    idx = np.flip( np.argsort(np.abs(eig)))
    eig = eig[idx]
    
    eig_1 = eig[0]
    eig_2 = eig[1]    
    
    energy = np.sum(eig**2)
    
    #Metrics related to normalized Laplacian    

    if library == 1:
        L_n = np.array( G.laplacian(normalized = True))
    else:
        L_n = nx.linalg.normalized_laplacian_matrix(G).toarray()        
    
    if not bool_eig_symm:
        eig_L_n = scipy.linalg.eig( L_n, right = False).astype(float)
    else:
        eig_L_n = scipy.linalg.eigh( L_n, eigvals_only = True)
    
#    num_zeros = np.sum( np.abs(eig_L_n)< eps)
    
    lower_slope = _get_slope( eig_L_n, low = 0, high = 1)
    num_ones = np.sum( np.abs(eig_L_n - 1)< eps)
    
    upper_slope = _get_slope( eig_L_n, low = 1, high = 2)
    
    num_twos = np.sum( np.abs(eig_L_n - 2)< eps) #L_n
    trace_L_n = np.trace(L_n)
    energy_L_n = np.sum(eig_L_n**2)
    
    return  {
        "eig_1": eig_1, 
        "eig_2": eig_2,
        "energy": energy, 
        "lower_slope": lower_slope,
        "num_ones": num_ones, 
        "upper_slope": upper_slope,
        "num_twos": num_twos, 
        "trace_L_n": trace_L_n,
        "energy_L_n": energy_L_n
    }

In [8]:
def fit_line( p1, p2):
    
    '''
    In:
        p1:     2,
        p2:     2,
    Test:
    
    fig, ax = plt.subplots()
    
    ax.plot( [p1[0],p2[0]],[p1[1],p2[1]],'*')
    
    x = np.arange(np.min( [p1[0],p2[0]]),
                  np.max( [p1[0],p2[0]]),100)
    
    y_est = x*m+c
    
    ax.plot( x, y_est, '.')
    '''
    
    X = np.vstack( [np.ones([2]),[p1[0],p2[0]]]).T
    y = np.hstack( [p1[1],p2[1]])
    
    c, m = lstsq( X, y, rcond = None)[0]
    
    return m, c

def fit_line_mul( X_in, y):
    
    '''
    In:
        X:  N,
        y:  N,
    '''
    
    X = np.vstack( [np.ones_like(X_in),X_in]).T
    
    c, m = lstsq( X, y, rcond = None)[0]
    
    return m, c

In [9]:
def getGraphFeats( G, library = 0, bool_spect = True):
    
    '''
    Extract the graph features.
    
    lcc stands for largest connected component
    
    In:
        G:              #Graph object should be consistent with library
        library:        #0: Uses networkx, 1: Uses igraph
        bool_spect:     #Turn on or off calculating spectral features
    
    TODO
        -Possible mismatch between libs in sparsely connected graphs.
        -Edge weights euclidean distance.
    '''        
        
    if library == 0:
        
        v_count = G.number_of_nodes()
        e_count = G.number_of_edges()
        
        #
        avg_clus = nx.average_clustering(G)                
            
        degree = np.array( G.degree(np.arange(v_count)))[:,1]
#        closeness = np.array( list(nx.closeness_centrality(G).items()))[:,1]
        
        #
        Gcc = sorted( nx.connected_components(G), key=len, reverse=True)
        G0 = G.subgraph(Gcc[0])     #Largest Connected Component
        
        e_count_G0 = G0.size()
        CC_count = len(Gcc)        
    
#        eccent_lcc = np.array( list(nx.eccentricity( G0).items()))[:,1]

#        try:
#            avg_short_path = nx.average_shortest_path_length(G)
#        except:
#            avg_short_path = np.nan
        
    elif library == 1:
        
        v_count = G.vcount()
        e_count = G.ecount()
        
        clus = np.array(G.transitivity_local_undirected())
        #If number of neighbors is zero or one, than clus=0
        clus[np.isnan(clus)] = 0
        avg_clus = np.average( clus)
        
        #
#        eccent = np.array( G.eccentricity())
        degree = np.array( G.degree())
#        closeness = np.array( G.closeness())
    
        # https://lists.nongnu.org/archive/html/igraph-help/2016-01/msg00015.html
        cl = G.clusters()
        G0 = cl.giant()
        
        e_count_G0 = G0.ecount()
        CC_count = len(cl.sizes())
        
#        eccent_lcc = np.array( G0.eccentricity())
        
        short_path_len = G.shortest_paths_dijkstra()
        avg_short_path = np.sum(short_path_len)/(v_count*(v_count-1))
        
        if np.isinf( avg_short_path):
            avg_short_path = np.nan
        
    eccent, eccent_90, avg_short_path = get_eccent(G, bool_get_90=True, verbose = True)
    
    avg_deg = np.average( degree)
    avg_eccent = np.average( eccent)
    diameter = np.max( eccent)
    radius = np.min( eccent)

    cent_p_count = np.where( eccent == radius)[0].shape[0]
    
    avg_eccent_90 = np.average( eccent_90)
    diameter_90 = np.max( eccent_90)
    radius_90 = np.min( eccent_90)
    
#    avg_closs = np.average( closeness)
    
    if v_count != 0 and e_count != 0:
        
        per_cent_points = cent_p_count/v_count*100
        
        ratio_size_LCC = e_count_G0/e_count    
        
        iso_v_count = np.where( degree == 0)[0].shape[0]
        perct_isolated = iso_v_count/v_count*100
        
        end_v_count = np.where( degree == 1)[0].shape[0]
        perct_end = end_v_count/v_count*100
        
    else:
        
        per_cent_points = np.nan
        ratio_size_LCC = np.nan
        
        perct_isolated = np.nan
        perct_end = np.nan
    
    
    #"Global Graph Features for Connectedness and Cliquishness Measures"    
    #available:  [1,2,12,13,14,15]
    #omitted:    [3,4]
    
    connectedness = {
        "avg_deg": avg_deg,
        "avg_clus": avg_clus,
        "ratio_size_LCC": ratio_size_LCC,
        "CC_count": CC_count, 
        "perct_isolated": perct_isolated,
        "perct_end": perct_end}
    
    #"Global Graph Features for Distance Based (Shortest-path related) Measures:"
    #available:  [5,6,7,8,9,10,11,16,17,18,19]
    #omitted:    []
    
    distance_measures = {
        "avg_eccent": avg_eccent,   
        "diameter": diameter,
        "radius": radius,       
        "avg_eccent_90": avg_eccent_90,
        "diameter_90": diameter_90,  
        "radius_90": radius_90,
        "avg_short_path": avg_short_path,
        "cent_p_count": cent_p_count,
        "per_cent_points": per_cent_points,
        "v_count": v_count,      
        "e_count": e_count
    }
    
    if bool_spect: 
        spectral_measures = get_feats_spect( G, library = library)
        
        return {
            "connectedness": connectedness,
            "distance_measures": distance_measures,
            "spectral_measures": spectral_measures
        }
    else:
        return {
            "connectedness": connectedness,
            "distance_measures": distance_measures
        } 

In [10]:
# Load the .mat file
mat = scipy.io.loadmat('gHoG/seed_coordinates.mat')

# Extract the list from the mat dictionary
seed_coordinates_all_images = mat['seed_coordinates'] 


In [12]:
for seeds_image in seed_coordinates_all_images:
    seeds_image = seeds_image[0]
    edge_list, dist_list = get_edge_list(seeds_image, 25)
    G = get_graph(edge_list)
    feats = getGraphFeats(G)
    print(feats)
    

{'connectedness': {'avg_deg': 5.517857142857143, 'avg_clus': 0.1650829081632653, 'ratio_size_LCC': 0.9870550161812298, 'CC_count': 7, 'perct_isolated': 0.0, 'perct_end': 0.0}, 'distance_measures': {'avg_eccent': 8.236607142857142, 'diameter': 11, 'radius': 0, 'avg_eccent_90': 6.483928571428572, 'diameter_90': 9.0, 'radius_90': 0.0, 'avg_short_path': inf, 'cent_p_count': 5, 'per_cent_points': 2.232142857142857, 'v_count': 224, 'e_count': 618}, 'spectral_measures': {'eig_1': 5.812860411968694, 'eig_2': 5.4492820542632785, 'energy': 1012.0000000000006, 'lower_slope': 0.007232010603277775, 'num_ones': 2, 'upper_slope': 0.0053791322372382004, 'num_twos': 0, 'trace_L_n': 164.53809523809525, 'energy_L_n': 161.27125661375652}}
{'connectedness': {'avg_deg': 14.52892561983471, 'avg_clus': 0.04461676222628225, 'ratio_size_LCC': 1.0, 'CC_count': 1, 'perct_isolated': 0.0, 'perct_end': 0.0}, 'distance_measures': {'avg_eccent': 4.08080808080808, 'diameter': 5, 'radius': 4, 'avg_eccent_90': 3.90615243